In [330]:
import requests, time
from bs4 import BeautifulSoup
ary = []
for page in range(1,47):
    res = requests.get('https://movies.yahoo.com.tw/movieinfo_review.html/id=6619?sort=update_ts&order=desc&page={}'.format(page))
    soup = BeautifulSoup(res.text, 'lxml')
    time.sleep(1)
    for li in soup.select('.usercom_list li'):
        stars = li.select_one('input[name="score"]').get('value')
        content = li.select('.form_good span')[2].text
        ary.append({'stars':stars, 'content': content, 'title':'最終騎士'})

In [331]:
import pandas
df = pandas.DataFrame(ary)
df.to_excel('最終騎士.xlsx')

In [332]:
a = pandas.read_excel('古墓奇兵.xlsx')
b = pandas.read_excel('最後的絕地武士.xlsx')
c = pandas.read_excel('攻殼機動隊.xlsx')
d = pandas.read_excel('奧創紀元.xlsx')
e = pandas.read_excel('最終騎士.xlsx')
df = pandas.concat([a,b,c,d,e])
#

In [404]:
df.to_excel('yahoo_movie.xlsx')

In [335]:
df['stars'] = df['stars'].astype(int)

In [336]:
df[df['stars'] <= 2].count()

content    428
stars      428
title      428
dtype: int64

In [337]:
df[df['stars'] >=4].count()

content    552
stars      552
title      552
dtype: int64

In [338]:
def getStatus(stars):
    if stars <= 2:
        return 'bad'
    elif stars == 3:
        return 'soso'
    else:
        return 'good'

In [339]:
df['status'] =  df['stars'].map(lambda e: getStatus(e))

In [340]:
df2 = df[df['status'] != 'soso']

In [341]:
import jieba
corpus = []
scores = []
for rec in df2.iterrows():
    corpus.append(' '.join(jieba.cut(rec[1].content)))
    scores.append(rec[1].status) 
#jieba.cut()

In [394]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = TfidfVectorizer(min_df=0.005)
vectorizer = TfidfVectorizer()
#?TfidfVectorizer(min_df=5)
#vectorizer = CountVectorizer(min_df=0.005)
X = vectorizer.fit_transform(corpus)

In [395]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, scores, test_size = 0.3, random_state = 42)

In [396]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.1)
clf.fit(train_X, train_Y)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [397]:
predicted = clf.predict(test_X)

In [400]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(test_Y, predicted )
#accuracy_score(test_Y, predicted)

array([[ 88,  29],
       [ 27, 150]])

In [402]:
import operator
coef_features_c1_c2 = []

for index, features in enumerate(zip(vectorizer.get_feature_names(), \
                        clf.feature_count_[0], clf.feature_count_[1])):
    feat,c1,c2 = features
    coef_features_c1_c2.append(tuple([c2/(c1 + 1), feat, c1, c2]))

for i in sorted(coef_features_c1_c2, key = operator.itemgetter(0), reverse=True)[0:20]:
    print(i)

(4.8179353022884825, '還不錯', 0.25067745442928002, 6.025683059471123)
(4.7792828636909803, '好看', 5.5477761086692068, 31.293674151447952)
(4.431739249825287, '不錯', 0.55837501936200729, 6.9063117392538498)
(3.2134490748808298, '非常', 0.8090592306180423, 5.8133197110341737)
(2.9850508304316468, '希望', 0.0, 2.9850508304316468)
(2.7880273224378231, '雖然', 0.69222512303168338, 4.7179698787280397)
(2.6423711140821093, '有些', 0.3279462446927951, 3.5089267980300547)
(2.64063655354777, '爽片', 1.2301739170550365, 5.8890787661443422)
(2.5105672307296309, '刺激', 0.47446888245356211, 3.701753259018453)
(2.3875622676102202, '不過', 0.52716553124341725, 3.6462027987917001)
(2.3818045781395365, '大家', 0.70280964780310939, 4.0557598148376179)
(2.3176102449698228, '感動', 0.0, 2.3176102449698228)
(2.164998097144236, '但是', 0.741982835065923, 3.7713895231756447)
(2.160756564443945, '其實', 0.62885409940606329, 3.5195571878130814)
(2.0885042689550155, '效果', 0.0, 2.0885042689550155)
(2.0195696651610677, '期待', 1.42960694899